In [1]:
!pip install deeplake

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/607.6 kB ? eta -:--:--
     ------------------------------------- 607.6/607.6 kB 13.0 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 57.6/57.6 kB ? eta 0:00:00
     ---------------------------------------- 0.0/125.2 kB ? eta -:--:--
     -------------------------------------- 125.2/125.2 kB 7.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   -------

In [5]:
!pip install -U deeplake

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/607.9 kB ? eta -:--:--
     ------------------------------------  604.2/607.9 kB 18.6 MB/s eta 0:00:01
     ------------------------------------- 607.9/607.9 kB 12.7 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for deeplake: filename=deeplake-3.9.15-py3-none-any.whl size=730461 sha256=e7d71fcb7f694aef5e429c3f9bc565dfc1f10724017061d2aa8e733dc84f776d
  Stored in directory: c:\users\anjal\appdata\local\pip\cache\wheels\ba\84\3a\d2f5ebe441d34ab8c165c85331110900162f47c2b4388ad71c
Successfully built deeplake
  Attempting uninstall: deeplake
    Found exi

In [6]:
import deeplake
import tensorflow as tf 
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import MaxPooling2D, Dense, Activation, Dropout, Conv2D, Flatten
from tensorflow.data import Dataset

In [2]:
train = deeplake.load('hub://activeloop/fer2013-train')

-

Opening dataset in read-only mode as you don't have write permissions.


/

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/fer2013-train



-

hub://activeloop/fer2013-train loaded successfully.



In [3]:
test = deeplake.load('hub://activeloop/fer2013-public-test')

|

Opening dataset in read-only mode as you don't have write permissions.


/

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/fer2013-public-test



\

hub://activeloop/fer2013-public-test loaded successfully.



In [4]:
validation = deeplake.load('hub://activeloop/fer2013-private-test')

/

Opening dataset in read-only mode as you don't have write permissions.


/

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/fer2013-private-test



/

hub://activeloop/fer2013-private-test loaded successfully.



In [7]:
def preprocess(images, labels):
    images = tf.cast(images, tf.float32) / 255.0  
    images = tf.reshape(images, [48, 48, 1])  
    labels= tf.one_hot(labels, 7)
    return images, labels

In [8]:
print(train.tensors)
print(validation.tensors)
print(test.tensors)

{'images': Tensor(key='images'), 'labels': Tensor(key='labels')}
{'images': Tensor(key='images'), 'labels': Tensor(key='labels')}
{'images': Tensor(key='images'), 'labels': Tensor(key='labels')}


In [9]:
def deeplake_to_tfds(deeplake_dataset):
    def generator():
        for sample in deeplake_dataset.pytorch():
            image = sample['images'].numpy().squeeze()  
            label = sample['labels'].numpy()
            if image.ndim == 2: 
                image = image.reshape((48, 48))  
            if label.ndim > 0:
                label = label.item()  
            yield image, label
    return tf.data.Dataset.from_generator(generator, output_signature=(
        tf.TensorSpec(shape=(48, 48), dtype=tf.uint8),
        tf.TensorSpec(shape=(), dtype=tf.int64)
    ))

In [10]:
train_tfds = deeplake_to_tfds(train)
val_tfds = deeplake_to_tfds(validation)
test_tfds = deeplake_to_tfds(test)

In [11]:
train_tfds = train_tfds.map(preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
val_tfds = val_tfds.map(preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_tfds = test_tfds.map(preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [12]:
batch_size = 64
train_tfds = train_tfds.shuffle(buffer_size=10000).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
val_tfds = val_tfds.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
test_tfds = test_tfds.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

In [16]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(7, activation='softmax')
])

In [17]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [18]:
 model.fit(train_tfds, epochs=10, batch_size=64, validation_data=val_tfds)

Epoch 1/10
    449/Unknown 181s 229ms/step - accuracy: 0.2849 - loss: 1.7586

C:\Program Files\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


449/449 ━━━━━━━━━━━━━━━━━━━━ 215s 305ms/step - accuracy: 0.2850 - loss: 1.7583 - val_accuracy: 0.4218 - val_loss: 1.5021
Epoch 2/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 171s 255ms/step - accuracy: 0.4517 - loss: 1.4336 - val_accuracy: 0.4868 - val_loss: 1.3342
Epoch 3/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 179s 262ms/step - accuracy: 0.5009 - loss: 1.3088 - val_accuracy: 0.5096 - val_loss: 1.2763
Epoch 4/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 184s 273ms/step - accuracy: 0.5447 - loss: 1.2145 - val_accuracy: 0.5255 - val_loss: 1.2306
Epoch 5/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 189s 280ms/step - accuracy: 0.5661 - loss: 1.1480 - val_accuracy: 0.5436 - val_loss: 1.1828
Epoch 6/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 179s 270ms/step - accuracy: 0.5962 - loss: 1.0802 - val_accuracy: 0.5517 - val_loss: 1.1611
Epoch 7/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 198s 278ms/step - accuracy: 0.6183 - loss: 1.0174 - val_accuracy: 0.5600 - val_loss: 1.1600
Epoch 8/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 261s 340ms/step - accuracy: 0.6434 - loss: 0.95

In [19]:
test_loss, test_acc = model.evaluate(test_tfds)
print(f'Test accuracy: {test_acc}')

57/57 ━━━━━━━━━━━━━━━━━━━━ 34s 167ms/step - accuracy: 0.5759 - loss: 1.2463
Test accuracy: 0.5592086911201477
